In [ ]:
import matplotlib.pyplot as plt
from ripser import ripser
from gudhi.point_cloud import timedelay
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
from persim import plot_diagrams
%matplotlib qt5

import plotly.graph_objects as go
import plotly.offline as py
from plotly.subplots import make_subplots
import plotly.io as pio

In [ ]:
def ACF(sig):
    t=int(len(sig)/2)
    corr=np.zeros(t)

    for index in np.arange(t):
        ACF_delay=sig[index:]
        L=(t-index)/2
        m = np.sum(sig[int(t-L):int(t+L+1)]**2) + np.sum(ACF_delay[int(t-L):int(t+L+1)]**2)
        r = np.sum(sig[int(t-L):int(t+L+1)]*ACF_delay[int(t-L):int(t+L+1)])
        corr[index] = 2*r/m

    #Step 2: Find the ''key max''
    #Compute zero crossings
    zc = np.zeros(corr.size-1)
    zc[(corr[0:-1] < 0)*(corr[1::] > 0)] = 1
    zc[(corr[0:-1] > 0)*(corr[1::] < 0)] = -1

    #Mark regions which are admissible for key maxes
    #(regions with positive zero crossing to left and negative to right)
    admiss = np.zeros(corr.size)
    admiss[0:-1] = zc
    for i in range(1, corr.size):
        if admiss[i] == 0:
            admiss[i] = admiss[i-1]

    #Find all local maxes
    maxes = np.zeros(corr.size)
    maxes[1:-1] = (np.sign(corr[1:-1] - corr[0:-2])==1)*(np.sign(corr[1:-1] - corr[2::])==1)
    maxidx = np.arange(corr.size)
    maxidx = maxidx[maxes == 1]
    max_index = 0
    if len(corr[maxidx]) > 0:
        max_index = maxidx[np.argmax(corr[maxidx])]

    return (max_index, corr)

In [ ]:
# how fast frequency/amplitude is varying can influence persistent diagram in a tractable way
t=np.arange(0,7*np.pi,0.01)
i=3
F=np.linspace(i/4,1,len(t))
#y=np.cos(F*t)#frequency
#y=F*np.cos(t)#amplitude
y=F+np.cos(t)#average line

M=3
delay=100
skip=10

point_Cloud=timedelay.TimeDelayEmbedding(M, delay, skip)
Points=point_Cloud(y)
dgms = ripser(Points)['dgms']
dgms=dgms[1]
persistent_time=[ele[1]-ele[0] for ele in dgms] 
BT=dgms[:,0]
LT=persistent_time
Big_Persistence=[ele for ele in range(len(LT)) if LT[ele]>0.1]
BT=np.delete(BT, Big_Persistence).tolist()
LT=np.delete(LT, Big_Persistence).tolist()

fig = go.Figure(data=go.Scatter(
    x=BT, y=LT,
    mode='markers',
    marker=dict(
        size=10,
        colorscale='Viridis',
    ),
))
fig.update_layout(
    title={},
    template="simple_white",
    xaxis_title="birth time",
    yaxis_title="lifetime",
)
fig.show()

In [ ]:
M=3
delay=100
skip=10
list=[1,2,3,4]
t=np.arange(0,7*np.pi,0.01)
point_Cloud=timedelay.TimeDelayEmbedding(M, delay, skip)
BT_list=[0,0,0,0]
LT_list=[0,0,0,0]
BT_trimed_list=[0,0,0,0]
LT_trimed_list=[0,0,0,0]
F=[0,0,0,0]
y=[0,0,0,0]

for i in list:
    F[i-1]=np.linspace(i/4,1,len(t))
    y[i-1]=F[i-1]+np.cos(t)#average line
    #y[i-1]=np.cos(F[i-1]*t)#frequency
    #y[i-1]=F[i-1]*np.cos(t)#amplitude

    Points=point_Cloud(y[i-1])
    dgms = ripser(Points)['dgms']
    dgms=dgms[1]
    persistent_time=[ele[1]-ele[0] for ele in dgms] 
    BT=dgms[:,0]
    BT_list[i-1]=BT
    LT=persistent_time
    LT_list[i-1]=LT

    Big_Persistence=[ele for ele in range(len(LT)) if LT[ele]>0.1]
    BT_trimed_list[i-1]=np.delete(BT, Big_Persistence).tolist()
    LT_trimed_list[i-1]=np.delete(LT, Big_Persistence).tolist()
    


In [ ]:
size=6
# color=['#006600','#00cc00','#66ff66',"#a3a375"]
# opacity=0.4
# color_opacity=['rgba(0, 102, 0, %f)'%opacity,'rgba(0, 204, 0, %f)'%opacity, 'rgba(102, 255, 102, %f)'%opacity, 'rgb(163, 163, 117, %f)'%opacity]
lable=['c = 1','c = 2', 'c = 3', 'c = 4']

color=['#ff6666','#809fff','#79d279',"#b2b266"]
opacity=0.6
color_opacity=['rgba(255, 102, 102, %f)'%opacity,'rgba(128, 159, 255, %f)'%opacity, 'rgba(121, 210, 121, %f)'%opacity, 'rgb(178, 178, 102, %f)'%opacity]


fig1 = go.Figure(data=[
    go.Scatter(
    x=BT_list[0], y=LT_list[0],
    mode='markers',
    name=lable[0],
    marker=dict(
        size=size,
        color=color[0],
        ),
    ), 
    go.Scatter(
    x=BT_list[1], y=LT_list[1],
    mode='markers',
    name=lable[1],
    marker=dict(
        size=size,
        color=color[1],
        ),
    ),
    go.Scatter(
    x=BT_list[2], y=LT_list[2],
    mode='markers',
    name=lable[2],
    marker=dict(
        size=size,
        color=color[2],
        ),
    ),
    go.Scatter(
    x=BT_list[3], y=LT_list[3],
    mode='markers',
    name=lable[3],
    marker=dict(
        size=size,
        color=color[3],
        ),
    ),
    ],
)

fig1.update_layout(
    width=500,
    title={},
    template="simple_white",
    xaxis_title="birth time",
    yaxis_title="lifetime",
    #showlegend=False,
)
fig1.show()
#pio.write_image(fig1, 'E:/Topological_Analysis_of_Time_Series/pictures/pdf_graph_raw/Intro_average_line1Color.pdf',scale=1)
#pio.write_image(fig1, 'E:/Topological_Analysis_of_Time_Series/pictures/pdf_graph_raw/Intro_frequency1Color.pdf',scale=1)
#pio.write_image(fig1, 'E:/Topological_Analysis_of_Time_Series/pictures/pdf_graph_raw/Intro_amplitude1Color.pdf',scale=1)
#pio.write_image(fig1, 'E:/Topological_Analysis_of_Time_Series/pictures/pdf_graph_raw/Intro_lable_infroColor.pdf',scale=1)


In [ ]:
BT_trimed_list[3]=BT_trimed_list[2]
LT_trimed_list[3]=LT_trimed_list[2]
color_opacity[3]=color_opacity[2]

fig = make_subplots(
    rows=2, cols=2,
    column_widths=[0.5, 0.5],
    row_heights=[0.5, 0.5],
    vertical_spacing=0.1,
    horizontal_spacing=0.1,
    specs=[[ {"type": "scatter"},{"type": "scatter"}],
           [ {"type": "scatter"},{"type": "scatter"}]])

for ele in [0,1,3]:
    fig.add_trace(
        go.Scatter(
        x=BT_trimed_list[ele], y=LT_trimed_list[ele], 
        mode='markers',
        
        marker=dict(
            size=10,
            color=color_opacity[ele],
            line=dict(width=1,
                color='DarkSlateGrey')
        ),
    ),
        row=int(ele/2)+1, col=ele%2+1
    )

fig.update_layout(
    width=700,
    height=500,
    margin=dict(r=1, t=1, b=1, l=1),
    showlegend=False,
)

#fig.update_traces(mode='markers', marker_line_width=.5)

fig.show()
pio.write_image(fig, 'Topological_Analysis_of_Time_Series/pictures/pdf_graph_raw/Intro_average_line2Color.pdf',scale=1)
#pio.write_image(fig, 'Topological_Analysis_of_Time_Series/pictures/pdf_graph_raw/Intro_frequency2Color.pdf',scale=1)
#pio.write_image(fig, 'Topological_Analysis_of_Time_Series/pictures/pdf_graph_raw/Intro_amplitude2Color.pdf',scale=1)
